# SOLUCIONES - SESIÓN 4: Testing e Integración con IA Generativa

**Estructura por ejercicio:**
1. Código resuelto con comentarios inline
2. Explicación pedagógica del "por qué"
3. Errores comunes y cómo evitarlos

**Nota:** Ejecuta las celdas en orden para que funcionen correctamente.

In [ ]:
# Instalación de dependencias
!pip install fastapi==0.115.0 uvicorn[standard]==0.32.0 pytest==8.0.0 httpx==0.27.0 openai==1.12.0 python-dotenv==1.0.0 -q
print("✅ Dependencias instaladas")

In [ ]:
# Imports necesarios
from fastapi import FastAPI, Depends, HTTPException, status
from fastapi.testclient import TestClient
from pydantic import BaseModel
from typing import Annotated, Optional
import asyncio
import time
from collections import defaultdict
from datetime import datetime, timedelta
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ Imports listos")

---

# 🧪 BLOQUE 1: TESTING (Ejercicios 1-5)

## EJERCICIO 1: Test de endpoint GET básico

**Objetivo:** Testear un endpoint GET que retorna información de un producto.

In [ ]:
# API simple con endpoint de producto
app_ej1 = FastAPI()

@app_ej1.get("/productos/{producto_id}")
def obtener_producto(producto_id: int):
    if producto_id == 1:
        return {"id": 1, "nombre": "Laptop", "precio": 999.99}
    elif producto_id == 2:
        return {"id": 2, "nombre": "Mouse", "precio": 25.50}
    else:
        raise HTTPException(status_code=404, detail="Producto no encontrado")

# SOLUCIÓN: Crear TestClient
# TestClient simula un navegador/cliente HTTP sin levantar un servidor real
client = TestClient(app_ej1)

# SOLUCIÓN: Hacer GET request
response = client.get("/productos/1")

# SOLUCIÓN: Verificaciones con assert
assert response.status_code == 200, "El status code debería ser 200 OK"

data = response.json()  # Convierte la respuesta JSON a diccionario Python
assert "nombre" in data, "La respuesta debe contener la clave 'nombre'"
assert data["nombre"] == "Laptop", "El nombre del producto debería ser 'Laptop'"

print("✅ Ejercicio 1 completado")
print(f"Respuesta obtenida: {data}")

### 📚 EXPLICACIÓN PEDAGÓGICA

**¿Qué es TestClient?**
- Es una clase de FastAPI que simula requests HTTP sin levantar un servidor real
- Internamente usa `httpx` (cliente HTTP asíncrono)
- Ventajas: tests rápidos (milisegundos), sin puertos ni procesos externos

**Patrón de testing (AAA):**
1. **Arrange** (Preparar): Crear cliente, definir datos de entrada
2. **Act** (Actuar): Ejecutar la acción (GET request)
3. **Assert** (Verificar): Comprobar resultados con asserts

**Status codes comunes:**
- 200 OK: Request exitoso (GET, PUT, PATCH)
- 201 Created: Recurso creado exitosamente (POST)
- 400 Bad Request: Error de validación
- 401 Unauthorized: Falta autenticación
- 404 Not Found: Recurso no existe
- 422 Unprocessable Entity: Error de validación Pydantic

**Errores comunes:**
- ❌ Olvidar `TestClient(app)` → `NameError: client not defined`
- ❌ No llamar a `.json()` → Intentar acceder a string como diccionario
- ❌ Comparar con `==` en vez de usar `in` para verificar claves → KeyError si la clave no existe

---

## EJERCICIO 2: Test de POST con validación Pydantic

**Objetivo:** Testear un endpoint POST que crea productos, verificando validación Pydantic.

In [ ]:
app_ej2 = FastAPI()

class ProductoCrear(BaseModel):
    nombre: str  # Campo obligatorio
    precio: float  # Campo obligatorio
    stock: int = 0  # Campo opcional con valor por defecto

@app_ej2.post("/productos", status_code=201)
def crear_producto(producto: ProductoCrear):
    return {"mensaje": "Producto creado", "producto": producto.dict()}

client_ej2 = TestClient(app_ej2)

# SOLUCIÓN Test 1: POST exitoso con datos completos
response_exitoso = client_ej2.post("/productos", json={
    "nombre": "Teclado",
    "precio": 49.99,
    "stock": 10
})

assert response_exitoso.status_code == 201, "POST exitoso debe retornar 201 Created"
print(f"✅ Test 1 pasó: {response_exitoso.json()}")

# SOLUCIÓN Test 2: POST sin campo obligatorio (nombre)
response_error = client_ej2.post("/productos", json={
    "precio": 29.99
    # Falta 'nombre' que es obligatorio
})

assert response_error.status_code == 422, "Falta de campo obligatorio debe retornar 422"
print(f"✅ Test 2 pasó: Error capturado correctamente")

# SOLUCIÓN Test 3: Verificar estructura del error 422
error_detail = response_error.json()["detail"]

# Pydantic retorna una lista de errores en detail
assert len(error_detail) > 0, "Debe haber al menos un error"

# Cada error tiene: type, loc (ubicación), msg
assert error_detail[0]["type"] == "missing", "El tipo de error debe ser 'missing'"
assert "nombre" in str(error_detail[0]["loc"]), "El error debe mencionar el campo 'nombre'"

print("✅ Ejercicio 2 completado")
print(f"Estructura de error: {error_detail[0]}")

### 📚 EXPLICACIÓN PEDAGÓGICA

**¿Por qué 422 y no 400?**
- FastAPI usa el estándar HTTP para APIs REST
- **400 Bad Request:** Error genérico del cliente
- **422 Unprocessable Entity:** La sintaxis es correcta, pero los datos no cumplen las reglas de validación
- Pydantic siempre retorna 422 cuando los datos no coinciden con el modelo

**Estructura del error 422:**
```json
{
  "detail": [
    {
      "type": "missing",
      "loc": ["body", "nombre"],
      "msg": "Field required"
    }
  ]
}
```

**Ventajas de Pydantic:**
- Validación automática de tipos (str, int, float, etc.)
- Mensajes de error claros y estructurados
- Conversión automática de tipos cuando es posible
- Valores por defecto (`stock: int = 0`)

**Errores comunes:**
- ❌ Usar `response.text` en vez de `response.json()` → Intentar indexar un string
- ❌ No verificar `len(error_detail) > 0` antes de acceder a `error_detail[0]` → IndexError
- ❌ Asumir que 422 es un "error" del código → Es el comportamiento esperado de validación

---

## EJERCICIO 3: Fixture de cliente de testing

**Objetivo:** Crear una función que actúe como fixture, retornando un cliente configurado.

In [ ]:
# SOLUCIÓN: Función fixture completa
def get_client_fixture():
    """Fixture que retorna un TestClient configurado
    
    Esta función encapsula la creación de la app y sus endpoints,
    permitiendo reutilizar la misma configuración en múltiples tests.
    """
    app = FastAPI()
    
    @app.get("/salud")
    def endpoint_salud():
        # Endpoint simple para health checks
        return {"status": "ok"}
    
    @app.get("/usuarios")
    def listar_usuarios():
        # Retorna una lista de usuarios fake
        return [
            {"id": 1, "nombre": "Ana"},
            {"id": 2, "nombre": "Juan"}
        ]
    
    # Retornar el cliente configurado
    return TestClient(app)

# SOLUCIÓN Test 1: Usar fixture para testear /salud
def test_salud():
    """Test del endpoint de health check"""
    client = get_client_fixture()  # Obtener cliente desde fixture
    response = client.get("/salud")
    
    assert response.status_code == 200, "Health check debe retornar 200"
    assert response.json() == {"status": "ok"}, "El JSON debe ser exactamente {status: ok}"
    
    print("✅ test_salud pasó")

# SOLUCIÓN Test 2: Usar fixture para testear /usuarios
def test_listar_usuarios():
    """Test del endpoint que lista usuarios"""
    client = get_client_fixture()  # Reutilizar la misma fixture
    response = client.get("/usuarios")
    
    assert response.status_code == 200, "Debe retornar 200"
    
    usuarios = response.json()
    assert len(usuarios) == 2, "Debe haber exactamente 2 usuarios"
    assert usuarios[0]["nombre"] == "Ana", "El primer usuario debe ser Ana"
    
    print("✅ test_listar_usuarios pasó")

# Ejecutar tests
test_salud()
test_listar_usuarios()
print("✅ Ejercicio 3 completado")

### 📚 EXPLICACIÓN PEDAGÓGICA

**¿Qué es una fixture?**
- Es una función que prepara el entorno necesario para los tests
- En pytest real, se usa el decorador `@pytest.fixture`
- Aquí simulamos el concepto con una función normal

**Ventajas de usar fixtures:**
1. **DRY (Don't Repeat Yourself):** No duplicar código de setup
2. **Aislamiento:** Cada test obtiene una instancia fresca
3. **Mantenibilidad:** Un cambio en la fixture afecta todos los tests
4. **Claridad:** Los tests se centran en la lógica, no en el setup

**Patrón de fixture con pytest real:**
```python
@pytest.fixture
def client():
    app = FastAPI()
    # ... configurar endpoints ...
    return TestClient(app)

def test_algo(client):  # pytest inyecta automáticamente
    response = client.get("/salud")
    assert response.status_code == 200
```

**Scope de fixtures en pytest:**
- `function` (default): Nueva instancia por cada test
- `module`: Una instancia compartida por todos los tests del módulo
- `session`: Una instancia para toda la sesión de tests

**Errores comunes:**
- ❌ No retornar el TestClient desde la fixture → `NoneType` error
- ❌ Modificar estado global dentro de la fixture → Tests no son independientes
- ❌ Fixtures muy complejas → Mejor dividir en fixtures más pequeñas

---

## EJERCICIO 4: Mockear dependencia de autenticación

**Objetivo:** Mockear una dependencia de autenticación para testear un endpoint protegido.

In [ ]:
app_ej4 = FastAPI()

class Usuario(BaseModel):
    username: str
    email: str

# Dependencia real (simula validar JWT)
def get_current_user() -> Usuario:
    """En producción: decodifica JWT, valida, busca en BD...
    
    Por ahora, simplemente lanza 401 para simular que no hay autenticación.
    """
    raise HTTPException(status_code=401, detail="No autenticado")

# Annotated type para inyección de dependencias
CurrentUser = Annotated[Usuario, Depends(get_current_user)]

@app_ej4.get("/mi-perfil")
def obtener_perfil(usuario: CurrentUser):
    """Endpoint protegido que requiere autenticación"""
    return {"username": usuario.username, "email": usuario.email}

# SOLUCIÓN: Función mock que retorna un usuario fake
def mock_get_current_user() -> Usuario:
    """Mock que bypasea la autenticación real
    
    En tests, no queremos validar JWTs reales.
    Este mock simplemente retorna un usuario predefinido.
    """
    return Usuario(username="testuser", email="test@example.com")

client_ej4 = TestClient(app_ej4)

# SOLUCIÓN Test 1: Sin mock, el endpoint debe retornar 401
response_sin_mock = client_ej4.get("/mi-perfil")
assert response_sin_mock.status_code == 401, "Sin autenticación debe retornar 401"
print("✅ Test 1 pasó: Endpoint protegido rechaza acceso sin auth")

# SOLUCIÓN Test 2: Con mock, el endpoint debe retornar 200
# Reemplazar la dependencia real con el mock
app_ej4.dependency_overrides[get_current_user] = mock_get_current_user

response_con_mock = client_ej4.get("/mi-perfil")
assert response_con_mock.status_code == 200, "Con mock debe retornar 200"

data = response_con_mock.json()
assert data["username"] == "testuser", "Debe retornar el usuario mockeado"
assert data["email"] == "test@example.com", "Debe retornar el email mockeado"

print("✅ Test 2 pasó: Mock de autenticación funciona correctamente")
print(f"Usuario obtenido: {data}")

# IMPORTANTE: Limpiar el override para no afectar otros tests
app_ej4.dependency_overrides = {}

print("✅ Ejercicio 4 completado")

### 📚 EXPLICACIÓN PEDAGÓGICA

**¿Por qué mockear dependencias?**
- En producción, `get_current_user()` haría operaciones lentas/complejas:
  - Decodificar JWT (criptografía)
  - Validar firma y expiración
  - Consultar base de datos
  - Verificar permisos
- En tests, queremos:
  - Velocidad (milisegundos, no segundos)
  - Aislamiento (sin BD externa)
  - Control (probar con usuarios específicos)

**¿Cómo funciona `dependency_overrides`?**
```python
app.dependency_overrides[funcion_real] = funcion_mock
```
- FastAPI internamente reemplaza todas las llamadas a `funcion_real` por `funcion_mock`
- Solo afecta al objeto `app` específico
- Es un diccionario: `{dependencia_original: dependencia_reemplazo}`

**Patrón completo de mock:**
1. Definir función mock con la misma firma de retorno
2. Establecer override ANTES del request
3. Hacer el request con TestClient
4. Limpiar override DESPUÉS del test (`app.dependency_overrides = {}`)

**Casos de uso comunes:**
- Mock de autenticación (este ejercicio)
- Mock de base de datos (retornar datos fake)
- Mock de APIs externas (simular responses)
- Mock de servicios lentos (emails, pagos, etc.)

**Errores comunes:**
- ❌ No limpiar `dependency_overrides` → Afecta otros tests
- ❌ Firma incorrecta del mock → TypeError en runtime
- ❌ Mockear en el lugar equivocado → Override no se aplica
- ❌ No usar `Depends()` en el endpoint → La dependencia no se inyecta

---

## EJERCICIO 5: Test de flujo completo (Login + Endpoint protegido)

**Objetivo:** Testear un sistema con login y endpoint protegido, usando el token obtenido.

In [ ]:
app_ej5 = FastAPI()

# Base de datos fake
USUARIOS_DB = {
    "admin": {
        "username": "admin",
        "password": "secret123",
        "email": "admin@example.com"
    }
}

class LoginRequest(BaseModel):
    username: str
    password: str

@app_ej5.post("/login")
def login(request: LoginRequest):
    """Endpoint de autenticación que valida credenciales"""
    user = USUARIOS_DB.get(request.username)
    
    # Validar que el usuario existe y la contraseña es correcta
    if not user or user["password"] != request.password:
        raise HTTPException(status_code=401, detail="Credenciales inválidas")
    
    # En producción: generar JWT real con pyjwt
    # Aquí: token fake para simplificar
    token = f"fake_token_for_{request.username}"
    return {"access_token": token, "token_type": "bearer"}

@app_ej5.get("/datos-privados")
def datos_privados(token: str = None):
    """Endpoint protegido que requiere token válido"""
    # Validación simple del token (en producción: decodificar JWT)
    if not token or not token.startswith("fake_token_for_"):
        raise HTTPException(status_code=401, detail="Token inválido")
    
    # Extraer username del token
    username = token.replace("fake_token_for_", "")
    return {"mensaje": f"Datos privados para {username}"}

client_ej5 = TestClient(app_ej5)

# SOLUCIÓN: Test del flujo completo
print("=== PASO 1: Login con credenciales correctas ===")
response_login = client_ej5.post("/login", json={
    "username": "admin",
    "password": "secret123"
})

assert response_login.status_code == 200, "Login exitoso debe retornar 200"
print(f"✅ Login exitoso: {response_login.json()}")

# Extraer el token de la respuesta
token = response_login.json()["access_token"]
assert token.startswith("fake_token_for_"), "Token debe tener el formato correcto"
print(f"Token obtenido: {token}")

print("\n=== PASO 2: Acceder a endpoint protegido CON token ===")
response_protegido = client_ej5.get(f"/datos-privados?token={token}")

assert response_protegido.status_code == 200, "Con token válido debe retornar 200"
data = response_protegido.json()
assert "admin" in data["mensaje"], "El mensaje debe mencionar al usuario 'admin'"
print(f"✅ Acceso permitido: {data}")

print("\n=== PASO 3: Intentar acceder SIN token ===")
response_sin_token = client_ej5.get("/datos-privados")

assert response_sin_token.status_code == 401, "Sin token debe retornar 401"
print(f"✅ Acceso denegado (como esperado): {response_sin_token.json()}")

print("\n✅ Ejercicio 5 completado: Flujo de autenticación funciona correctamente")

### 📚 EXPLICACIÓN PEDAGÓGICA

**Flujo de autenticación típico:**
```
1. Cliente envía credenciales → POST /login
2. Servidor valida y retorna token → {access_token: "..."}
3. Cliente guarda token (localStorage, cookie, etc.)
4. Cliente usa token en requests posteriores → Authorization: Bearer ...
5. Servidor valida token en cada request
```

**¿Por qué tokens en vez de username/password cada vez?**
- **Seguridad:** No enviar contraseña en cada request
- **Performance:** Validar token es más rápido que validar contraseña
- **Stateless:** El servidor no necesita mantener sesiones
- **Expiración:** Los tokens pueden caducar automáticamente
- **Revocación:** Puedes invalidar tokens específicos

**Diferencias entre token fake y JWT real:**

| Aspecto | Token Fake | JWT Real |
|---------|-----------|----------|
| Formato | String simple | Base64 con 3 partes (header.payload.signature) |
| Seguridad | ⚠️ No seguro | ✅ Firma criptográfica |
| Contenido | Solo identificador | Claims (user_id, roles, exp, etc.) |
| Validación | Simple string check | Verificar firma + expiración |
| Uso | Solo tests | Producción |

**Ejemplo de JWT real:**
```python
import jwt
from datetime import datetime, timedelta

payload = {
    "sub": "admin",
    "exp": datetime.utcnow() + timedelta(hours=1)
}
token = jwt.encode(payload, "secret_key", algorithm="HS256")
# token = "eyJ0eXAiOi..."
```

**Patrón de testing de flujos:**
1. Dividir el flujo en pasos claros
2. Hacer asserts después de cada paso
3. Usar las respuestas de pasos anteriores en pasos siguientes
4. Testear tanto el "happy path" como casos de error

**Errores comunes:**
- ❌ No extraer el token correctamente → KeyError o None
- ❌ Olvidar f-string en el query param → `/datos-privados?token=token` (literal)
- ❌ No testear el caso sin token → Código vulnerable a acceso no autorizado

---

# 🤖 BLOQUE 2: INTEGRACIÓN CON IA GENERATIVA (Ejercicios 6-10)

## EJERCICIO 6: Llamada básica a API de IA (completion)

**Objetivo:** Completar una función que simula una llamada a OpenAI.

In [ ]:
# Mock de respuesta de OpenAI (para tests sin API key real)
class MockChoice:
    """Simula la estructura de una opción de respuesta de OpenAI"""
    def __init__(self, content):
        # OpenAI retorna: response.choices[0].message.content
        self.message = type('obj', (object,), {'content': content})

class MockResponse:
    """Simula la estructura completa de una respuesta de OpenAI"""
    def __init__(self, content, tokens_used):
        self.choices = [MockChoice(content)]
        
        # Simula la estructura de usage
        self.usage = type('obj', (object,), {
            'prompt_tokens': 10,  # Tokens del prompt
            'completion_tokens': tokens_used,  # Tokens de la respuesta
            'total_tokens': 10 + tokens_used  # Total
        })

# SOLUCIÓN: Función completa con async
async def llamar_ia(prompt: str, max_tokens: int = 100):
    """Simula llamada a OpenAI (asíncrona)
    
    En producción, esto sería algo como:
    response = await openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[...],
        max_tokens=max_tokens
    )
    """
    
    # Estructura de mensajes (system + user)
    messages = [
        {"role": "system", "content": "Eres un asistente útil"},
        {"role": "user", "content": prompt}
    ]
    
    # Simulación de latencia de red (100ms)
    await asyncio.sleep(0.1)
    
    # Simulación de respuesta
    respuesta_texto = f"Respuesta simulada a: '{prompt}'"
    
    # Retornar mock con estructura de OpenAI
    return MockResponse(respuesta_texto, tokens_used=20)

# SOLUCIÓN: Test de la función
response = await llamar_ia("¿Qué es FastAPI?")

# Verificar estructura de la respuesta
contenido = response.choices[0].message.content
assert "FastAPI" in contenido, "La respuesta debe mencionar 'FastAPI'"
print(f"✅ Respuesta obtenida: {contenido}")

# Verificar tokens
assert response.usage.total_tokens == 30, "Total debe ser 10 (prompt) + 20 (completion)"
print(f"✅ Tokens: prompt={response.usage.prompt_tokens}, completion={response.usage.completion_tokens}, total={response.usage.total_tokens}")

print("✅ Ejercicio 6 completado")

### 📚 EXPLICACIÓN PEDAGÓGICA

**Estructura de una llamada a OpenAI:**
```python
from openai import AsyncOpenAI

client = AsyncOpenAI(api_key="sk-...")

response = await client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Eres un asistente"},
        {"role": "user", "content": "Hola"}
    ],
    max_tokens=100,
    temperature=0.7
)

texto = response.choices[0].message.content
```

**Roles de mensajes:**
- **system:** Instrucciones globales que definen el comportamiento del asistente
- **user:** Mensajes del usuario (tú)
- **assistant:** Mensajes previos del asistente (para mantener contexto)

**Parámetros importantes:**
- `model`: Qué modelo usar (gpt-4, gpt-3.5-turbo, claude-3-opus, etc.)
- `max_tokens`: Límite de tokens en la respuesta
- `temperature` (0-2): Creatividad (0=determinista, 2=muy creativo)
- `top_p` (0-1): Alternativa a temperature

**¿Qué son los tokens?**
- Fragmentos de texto que el modelo procesa
- 1 token ≈ 0.75 palabras en inglés
- Ejemplos:
  - "Hola" = 1 token
  - "FastAPI" = 2 tokens (Fast + API)
  - "¿Qué es esto?" = 5 tokens

**¿Por qué async?**
- Las llamadas a APIs externas son operaciones de I/O (red)
- `async/await` permite que el servidor atienda otros requests mientras espera la respuesta
- Sin async: Un request que tarda 3s bloquea el servidor
- Con async: El servidor puede atender 100+ requests en esos 3s

**Errores comunes:**
- ❌ No usar `await` → Recibir coroutine en vez de resultado
- ❌ Olvidar `async def` → `SyntaxError: 'await' outside async function`
- ❌ Acceder a índices incorrectos → `response.choices[0]` es un array
- ❌ No manejar casos donde `choices` está vacío → IndexError

---

## EJERCICIO 7: Endpoint POST /completar con prompt

**Objetivo:** Crear un endpoint que recibe un prompt y retorna la respuesta de la IA.

⚠️ **Nota:** Este ejercicio usa la función `llamar_ia()` del Ejercicio 6.

In [ ]:
app_ej7 = FastAPI()

# SOLUCIÓN: Modelos Pydantic
class CompletionRequest(BaseModel):
    """Request que el usuario envía al endpoint"""
    prompt: str  # El texto que quiere enviar a la IA
    max_tokens: int = 100  # Límite de tokens (default: 100)

class CompletionResponse(BaseModel):
    """Response que el endpoint retorna al usuario"""
    respuesta: str  # El texto generado por la IA
    tokens_usados: int  # Cuántos tokens consumió

# SOLUCIÓN: Endpoint completo
@app_ej7.post("/completar", response_model=CompletionResponse)
async def completar(request: CompletionRequest):
    """Endpoint que expone la funcionalidad de IA
    
    Args:
        request: Prompt y parámetros de la llamada
    
    Returns:
        CompletionResponse con la respuesta y tokens usados
    """
    # Llamar a la función de IA (recuerda usar await)
    response = await llamar_ia(request.prompt, request.max_tokens)
    
    # Extraer datos relevantes de la respuesta
    texto = response.choices[0].message.content
    tokens = response.usage.total_tokens
    
    # Retornar como CompletionResponse
    return CompletionResponse(
        respuesta=texto,
        tokens_usados=tokens
    )

client_ej7 = TestClient(app_ej7)

# SOLUCIÓN: Test del endpoint
response = client_ej7.post("/completar", json={
    "prompt": "Explica Python",
    "max_tokens": 50
})

assert response.status_code == 200, "Request válido debe retornar 200"
print(f"✅ Status code: {response.status_code}")

data = response.json()
assert "respuesta" in data, "La respuesta debe tener la clave 'respuesta'"
assert "tokens_usados" in data, "La respuesta debe tener la clave 'tokens_usados'"
print(f"✅ Estructura correcta: {list(data.keys())}")

assert data["tokens_usados"] == 30, "Tokens debe ser 30 (10 prompt + 20 completion)"
print(f"✅ Tokens: {data['tokens_usados']}")

print(f"\n📝 Respuesta de la IA: {data['respuesta']}")
print("✅ Ejercicio 7 completado")

### 📚 EXPLICACIÓN PEDAGÓGICA

**¿Por qué usar Pydantic para request y response?**
- **Validación automática:** FastAPI valida que el JSON enviado coincida con CompletionRequest
- **Documentación:** Los modelos se muestran automáticamente en Swagger UI
- **Serialización:** Pydantic convierte objetos Python a JSON y viceversa
- **Type hints:** Los IDEs dan autocompletado y detectan errores

**Flujo completo de un request POST:**
```
1. Cliente envía JSON:
   POST /completar
   {"prompt": "Hola", "max_tokens": 50}

2. FastAPI valida con Pydantic:
   ✓ prompt es str? Sí
   ✓ max_tokens es int? Sí
   → Crea objeto CompletionRequest

3. Endpoint llama a IA:
   response = await llamar_ia(...)

4. Endpoint crea respuesta:
   return CompletionResponse(...)

5. FastAPI serializa a JSON:
   {"respuesta": "...", "tokens_usados": 30}

6. Cliente recibe JSON
```

**Ventajas de `response_model`:**
```python
@app.post("/completar", response_model=CompletionResponse)
```
- FastAPI valida que tu función retorna el tipo correcto
- Filtra campos extra (solo retorna los definidos en CompletionResponse)
- Genera JSON Schema automático para la documentación

**Patrón de diseño: Request/Response Objects**
```
Cliente → [Request Object] → Endpoint → [Business Logic] → [Response Object] → Cliente
```
- Separa la lógica de validación de la lógica de negocio
- Los objetos son reutilizables (ej: en otros endpoints)
- Cambios en el modelo no afectan la lógica interna

**Errores comunes:**
- ❌ Olvidar `await` → `TypeError: object dict can't be used in 'await' expression`
- ❌ No usar `response_model` → FastAPI retorna cualquier estructura
- ❌ Retornar dict en vez de objeto Pydantic → Funciona pero pierde validación

---

## EJERCICIO 8: Control de max_tokens

**Objetivo:** Añadir validación de max_tokens a un endpoint.

In [ ]:
app_ej8 = FastAPI()

class IARequest(BaseModel):
    prompt: str
    max_tokens: int = 100

# SOLUCIÓN: Endpoint con validación de max_tokens
@app_ej8.post("/completar-seguro")
async def completar_seguro(request: IARequest):
    """Endpoint con límite de tokens para evitar costes excesivos
    
    En producción, esto protege contra:
    - Usuarios maliciosos que piden 10,000 tokens
    - Bugs en el frontend que envían valores incorrectos
    - Costes inesperados en la factura de OpenAI
    """
    
    # VALIDACIÓN: max_tokens debe ser <= 500
    if request.max_tokens > 500:
        raise HTTPException(
            status_code=400,
            detail="max_tokens excede límite de 500"
        )
    
    # Si pasa la validación, proceder normalmente
    response = await llamar_ia(request.prompt, request.max_tokens)
    
    # Retornar respuesta
    return {
        "respuesta": response.choices[0].message.content,
        "tokens_usados": response.usage.total_tokens
    }

client_ej8 = TestClient(app_ej8)

# SOLUCIÓN Test 1: max_tokens dentro del límite (100)
print("=== Test 1: Request válido (max_tokens=100) ===")
response_ok = client_ej8.post("/completar-seguro", json={
    "prompt": "Test",
    "max_tokens": 100
})

assert response_ok.status_code == 200, "Request válido debe retornar 200"
print(f"✅ Status: {response_ok.status_code}")
print(f"✅ Respuesta: {response_ok.json()}")

# SOLUCIÓN Test 2: max_tokens excede el límite (1000)
print("\n=== Test 2: Request inválido (max_tokens=1000) ===")
response_error = client_ej8.post("/completar-seguro", json={
    "prompt": "Test",
    "max_tokens": 1000
})

assert response_error.status_code == 400, "Request con max_tokens > 500 debe retornar 400"
print(f"✅ Status: {response_error.status_code}")

error_detail = response_error.json()["detail"]
assert "excede límite" in error_detail, "El mensaje debe mencionar 'excede límite'"
print(f"✅ Mensaje de error: {error_detail}")

print("\n✅ Ejercicio 8 completado")

### 📚 EXPLICACIÓN PEDAGÓGICA

**¿Por qué limitar max_tokens?**
- **Costes:** GPT-4 puede costar $0.06 por 1K tokens
  - 500 tokens = $0.03
  - 10,000 tokens = $0.60
  - 1,000 requests de 10K tokens = $600
- **Performance:** Respuestas más largas tardan más
- **UX:** Respuestas de 10K tokens son difíciles de leer
- **Seguridad:** Previene ataques de denegación de servicio (DoS)

**¿Validación en Pydantic vs en el endpoint?**

**Opción 1: Validación en Pydantic**
```python
from pydantic import Field

class IARequest(BaseModel):
    max_tokens: int = Field(le=500)  # le = less or equal
```
- ✅ Más limpio (validación declarativa)
- ✅ Pydantic genera error 422 automáticamente
- ❌ Mensaje de error genérico

**Opción 2: Validación en el endpoint (la que usamos)**
```python
if request.max_tokens > 500:
    raise HTTPException(400, "max_tokens excede límite")
```
- ✅ Mensaje de error personalizado
- ✅ Lógica de validación compleja (ej: límites por tier de usuario)
- ✅ Control del status code (400 vs 422)
- ❌ Más código en el endpoint

**Cuándo usar cada opción:**
- Pydantic Field: Validaciones simples de tipos/rangos
- Endpoint: Validaciones de negocio complejas

**Patrón de validación en capas:**
```
Request → [Validación Pydantic] → [Validación de negocio] → [Lógica del endpoint]
              (tipos, formato)      (límites, permisos)        (procesamiento)
```

**Errores comunes:**
- ❌ Usar status 422 en vez de 400 → 422 es para errores de Pydantic, 400 para errores de negocio
- ❌ No testear el caso límite exacto (500) → Falla cuando max_tokens=500
- ❌ Validar después de llamar a la IA → Ya gastaste tokens innecesariamente

---

## EJERCICIO 9: Manejo de errores y timeouts

**Objetivo:** Implementar llamada a IA con reintentos automáticos.

In [ ]:
# Contador de intentos (para simular fallos)
intentos_realizados = 0

def llamar_ia_inestable(prompt: str):
    """Simula API inestable que falla 2 veces antes de funcionar
    
    Esto simula errores reales como:
    - 500 Internal Server Error
    - 503 Service Unavailable
    - TimeoutError
    - ConnectionError
    """
    global intentos_realizados
    intentos_realizados += 1
    
    # Fallar los primeros 2 intentos
    if intentos_realizados < 3:
        raise Exception(f"Error temporal de API (intento {intentos_realizados})")
    
    # El tercer intento funciona
    return MockResponse(f"Respuesta a: {prompt}", tokens_used=15)

# SOLUCIÓN: Función con retry logic completa
async def llamar_ia_con_retry(prompt: str, max_intentos: int = 3):
    """Llama a IA con reintentos y backoff exponencial
    
    Backoff exponencial: 2^0=1s, 2^1=2s, 2^2=4s, 2^3=8s...
    Esto evita sobrecargar la API cuando está teniendo problemas.
    
    Args:
        prompt: Texto a enviar a la IA
        max_intentos: Número máximo de reintentos
    
    Returns:
        MockResponse si tiene éxito
    
    Raises:
        HTTPException 503 si fallan todos los intentos
    """
    
    for intento in range(max_intentos):
        try:
            # Intentar llamar a la API
            response = llamar_ia_inestable(prompt)
            
            # Si llegamos aquí, la llamada fue exitosa
            print(f"✅ Éxito en intento {intento + 1}")
            return response
        
        except Exception as e:
            # Calcular tiempo de espera con backoff exponencial
            espera = 2 ** intento  # 1s, 2s, 4s, 8s...
            
            print(f"❌ Intento {intento + 1} falló: {e}")
            
            # Si NO es el último intento, esperar y reintentar
            if intento < max_intentos - 1:
                print(f"⏳ Esperando {espera}s antes de reintentar...")
                await asyncio.sleep(espera)
            else:
                # Si ES el último intento, lanzar error final
                print(f"💥 Fallaron todos los {max_intentos} intentos")
                raise HTTPException(
                    status_code=503,
                    detail="Servicio de IA no disponible tras múltiples intentos"
                )

# SOLUCIÓN: Test de la función
print("=== Iniciando test de retry logic ===")
intentos_realizados = 0  # Resetear contador

response = await llamar_ia_con_retry("Test retry")

# Verificaciones
assert intentos_realizados == 3, "Debe haber hecho exactamente 3 intentos"
print(f"\n✅ Número de intentos correcto: {intentos_realizados}")

assert "Test retry" in response.choices[0].message.content, "La respuesta debe contener el prompt"
print(f"✅ Respuesta correcta: {response.choices[0].message.content}")

print("\n✅ Ejercicio 9 completado")

### 📚 EXPLICACIÓN PEDAGÓGICA

**¿Por qué usar retry logic?**
- Las APIs externas pueden fallar temporalmente por:
  - Sobrecarga de servidores (muchos requests simultáneos)
  - Problemas de red (latencia, pérdida de paquetes)
  - Mantenimiento o deploys
  - Rate limiting (excediste el límite de requests)
- El retry automático mejora la **resiliencia** del sistema

**¿Qué es backoff exponencial?**
- **Problema:** Si 1000 clientes reintentan inmediatamente, colapsan aún más el servidor
- **Solución:** Esperar más tiempo entre cada reintento

**Comparación de estrategias:**

| Estrategia | Intento 1 | Intento 2 | Intento 3 | Total tiempo |
|------------|-----------|-----------|-----------|---------------|
| Sin espera | 0s | 0s | 0s | 0s |
| Espera fija (1s) | 0s | 1s | 2s | 3s |
| Exponencial (2^n) | 0s | 1s | 3s | 4s |
| Exponencial (2^n) 5 intentos | 0s | 1s | 3s, 7s, 15s | 31s |

**Patrón completo de retry con todas las mejores prácticas:**
```python
MAX_INTENTOS = 3
BACKOFF_BASE = 2
JITTER = 0.1  # Aleatorización para evitar "thundering herd"

for intento in range(MAX_INTENTOS):
    try:
        return await llamar_api()
    except TemporaryError as e:
        if intento == MAX_INTENTOS - 1:
            raise FinalError("Fallaron todos los intentos")
        
        # Backoff exponencial con jitter
        espera = (BACKOFF_BASE ** intento) * (1 + random.uniform(-JITTER, JITTER))
        await asyncio.sleep(espera)
```

**¿Cuándo NO usar retry?**
- Errores 4xx (excepto 429 Too Many Requests)
  - 400 Bad Request → El request está mal, reintentar no ayuda
  - 401 Unauthorized → Falta autenticación, reintentar no ayuda
  - 404 Not Found → El recurso no existe, reintentar no ayuda
- Operaciones no idempotentes sin control
  - POST que crea recursos → Podrías crear duplicados
  - Pagos → Podrías cobrar múltiples veces

**Status codes y retry:**
- ✅ Reintentar: 500, 502, 503, 504, 429
- ❌ No reintentar: 400, 401, 403, 404, 422

**Errores comunes:**
- ❌ Reintentar inmediatamente → Colapsar el servidor
- ❌ Reintentar infinitamente → El sistema nunca falla, solo se queda colgado
- ❌ No usar backoff exponencial → Todos reintentan a la vez ("thundering herd")
- ❌ Reintentar errores 4xx → Desperdiciar recursos en requests que nunca funcionarán

---

## EJERCICIO 10: Sistema completo - Auth + IA + Rate limiting + Logging

**Objetivo:** Construir un endpoint de IA con todas las protecciones y mejores prácticas.

In [ ]:
app_ej10 = FastAPI()

# Rate limiting store (en producción: usar Redis)
rate_limit_store = defaultdict(list)

# ✅ FUNCIÓN YA IMPLEMENTADA
def estimar_coste(prompt_tokens: int, completion_tokens: int) -> float:
    """Calcula coste en USD (GPT-3.5-turbo: $0.50 input, $1.50 output por 1M tokens)"""
    coste_input = (prompt_tokens * 0.50) / 1_000_000
    coste_output = (completion_tokens * 1.50) / 1_000_000
    return coste_input + coste_output

# SOLUCIÓN: check_rate_limit completa
def check_rate_limit(user_id: str, max_requests: int = 3) -> bool:
    """Verifica si el usuario excedió el límite (ventana de 1 minuto)
    
    Esta función implementa una ventana deslizante:
    - Solo cuenta requests de los últimos 60 segundos
    - Los requests más antiguos se descartan automáticamente
    
    Args:
        user_id: Identificador del usuario
        max_requests: Número máximo de requests por minuto
    
    Returns:
        True si el usuario puede hacer el request, False si excedió el límite
    """
    now = datetime.now()
    window_start = now - timedelta(minutes=1)
    
    # Filtrar solo los requests DENTRO de la ventana temporal
    recent_requests = [
        req for req in rate_limit_store[user_id]
        if req > window_start
    ]
    
    # Actualizar el store con solo los requests recientes
    # (Esto descarta automáticamente requests antiguos)
    rate_limit_store[user_id] = recent_requests
    
    # Verificar si excedió el límite
    if len(recent_requests) >= max_requests:
        return False
    
    # Si no excedió, registrar el request actual y permitir
    rate_limit_store[user_id].append(now)
    return True

# Dependencia mockeada de auth
def mock_get_user():
    return {"user_id": "testuser", "username": "Test User"}

# SOLUCIÓN: Endpoint completo con todas las mejores prácticas
@app_ej10.post("/ia/completar")
async def completar_ia(
    request: IARequest,
    user=Depends(mock_get_user)
):
    """Endpoint de producción con auth, rate limit, IA, y logging
    
    Este endpoint integra todas las mejores prácticas:
    1. Autenticación (Depends)
    2. Rate limiting (3 req/min por usuario)
    3. Validación de parámetros (max_tokens <= 500)
    4. Logging estructurado (request + response)
    5. Llamada a IA con retry (del Ejercicio 9)
    6. Cálculo de costes
    7. Métricas de performance (duración)
    """
    start_time = time.time()
    user_id = user["user_id"]
    
    # 1. VERIFICAR RATE LIMIT
    if not check_rate_limit(user_id, max_requests=3):
        logger.warning(f"[RATE LIMIT] user={user_id} excedió límite")
        raise HTTPException(
            status_code=429,
            detail="Rate limit excedido. Intenta en 1 minuto."
        )
    
    # 2. VALIDAR MAX_TOKENS
    if request.max_tokens > 500:
        raise HTTPException(
            status_code=400,
            detail="max_tokens excede límite de 500"
        )
    
    # 3. LOG DEL REQUEST
    logger.info(
        f"[IA REQUEST] user={user_id} "
        f"prompt_len={len(request.prompt)} "
        f"max_tokens={request.max_tokens}"
    )
    
    # 4. LLAMAR A IA (con await)
    response = await llamar_ia(request.prompt, request.max_tokens)
    
    # 5. CALCULAR MÉTRICAS
    tokens_used = response.usage.total_tokens
    coste = estimar_coste(
        response.usage.prompt_tokens,
        response.usage.completion_tokens
    )
    duracion_ms = int((time.time() - start_time) * 1000)
    
    # 6. LOG DEL RESPONSE
    logger.info(
        f"[IA RESPONSE] user={user_id} "
        f"tokens={tokens_used} "
        f"coste=${coste:.6f} "
        f"duracion={duracion_ms}ms"
    )
    
    # 7. RETORNAR RESPUESTA CON MÉTRICAS
    return {
        "respuesta": response.choices[0].message.content,
        "tokens_usados": tokens_used,
        "coste_estimado": coste,
        "duracion_ms": duracion_ms
    }

print("✅ Endpoint completo definido")

In [ ]:
# SOLUCIÓN: Tests del sistema completo
client_ej10 = TestClient(app_ej10)
rate_limit_store.clear()  # Limpiar para tests frescos

print("=== Test 1: Request normal exitoso ===")
response1 = client_ej10.post("/ia/completar", json={
    "prompt": "Test 1",
    "max_tokens": 50
})

assert response1.status_code == 200, "Request válido debe retornar 200"
data1 = response1.json()
assert "respuesta" in data1, "Debe tener clave 'respuesta'"
assert "tokens_usados" in data1, "Debe tener clave 'tokens_usados'"
assert "coste_estimado" in data1, "Debe tener clave 'coste_estimado'"
assert "duracion_ms" in data1, "Debe tener clave 'duracion_ms'"
print(f"✅ Test 1 pasó. Métricas: {data1}")

print("\n=== Test 2: Exceder max_tokens ===")
response2 = client_ej10.post("/ia/completar", json={
    "prompt": "Test 2",
    "max_tokens": 1000
})

assert response2.status_code == 400, "max_tokens > 500 debe retornar 400"
print(f"✅ Test 2 pasó. Error: {response2.json()['detail']}")

print("\n=== Test 3: Exceder rate limit (hacer 4 requests) ===")
# Ya hicimos 1 request en Test 1, necesitamos 3 más para llegar al límite
for i in range(4):
    response = client_ej10.post("/ia/completar", json={
        "prompt": f"Test {i+3}",
        "max_tokens": 50
    })
    
    # Primeros 2 requests después del Test 1 = 3 requests totales (dentro del límite)
    if i < 2:
        assert response.status_code == 200, f"Request {i+2}/3 debe funcionar"
        print(f"✅ Request {i+2}/3: 200 OK")
    else:
        # Request 4+ debe fallar (rate limit excedido)
        assert response.status_code == 429, f"Request {i+2} debe retornar 429"
        print(f"✅ Request {i+2}: 429 Rate Limit (esperado)")

print("\n✅ Ejercicio 10 completado")
print("🎉 ¡Felicitaciones! Has completado todos los ejercicios de Testing e IA")

### 📚 EXPLICACIÓN PEDAGÓGICA FINAL

**¿Por qué este endpoint es "production-ready"?**

**1. Autenticación (Depends)**
- Cada request está asociado a un usuario
- Permite auditoría y control de acceso
- En producción: validar JWT real

**2. Rate Limiting**
- Previene abuso (usuarios maliciosos)
- Protege contra bugs del frontend (loops infinitos)
- Distribuye recursos equitativamente
- En producción: usar Redis para compartir estado entre servidores

**3. Validación de negocio**
- Límites de tokens previenen costes excesivos
- Verificación antes de llamar a la API (ahorro de dinero)
- Mensajes de error claros para debugging

**4. Logging estructurado**
- **Request logs:** Auditoría de quién usa el sistema
- **Response logs:** Métricas de costes y performance
- **Error logs:** Debugging cuando algo falla
- En producción: enviar a ELK Stack, Datadog, etc.

**5. Métricas de negocio**
- **Tokens:** Para facturación y optimización
- **Coste:** Tracking de gastos por usuario/feature
- **Duración:** Detectar bottlenecks de performance

**Arquitectura completa de un sistema de IA:**
```
Cliente → [Load Balancer] → [API Gateway]
                               ↓
                    [Auth Middleware]
                               ↓
                    [Rate Limiter (Redis)]
                               ↓
                    [Business Logic]
                               ↓
                    [LLM API (OpenAI)]
                               ↓
                    [Logging (ELK)]
                               ↓
                    [Métricas (Datadog)]
```

**Mejoras adicionales para producción:**
1. **Caché de respuestas:** Si el mismo prompt se repite, retornar desde caché
2. **Circuit breaker:** Si OpenAI falla mucho, dejar de intentar temporalmente
3. **Fallback models:** Si GPT-4 falla, usar GPT-3.5-turbo
4. **Request queuing:** Encolar requests en RabbitMQ para procesamiento asíncrono
5. **A/B testing:** Probar diferentes prompts con % de usuarios
6. **Feature flags:** Activar/desactivar features sin redeploy

**Comparación: Endpoint simple vs Endpoint de producción**

| Aspecto | Simple | Producción |
|---------|--------|------------|
| Auth | ❌ | ✅ Depends |
| Rate limit | ❌ | ✅ 3 req/min |
| Validación | ❌ | ✅ max_tokens |
| Logging | ❌ | ✅ Estructurado |
| Métricas | ❌ | ✅ Tokens/coste/duración |
| Retry | ❌ | ✅ Backoff exponencial |
| Errores | ❌ | ✅ HTTP codes correctos |
| Tests | ❌ | ✅ Happy path + errores |

**Lecciones clave de este ejercicio:**
1. **Diseñar para fallos:** Asumir que todo puede fallar
2. **Observabilidad:** Si no lo puedes medir, no lo puedes mejorar
3. **Defensa en profundidad:** Múltiples capas de validación
4. **Costes importan:** Cada token cuesta dinero real
5. **UX es clave:** Mensajes de error claros ayudan al debugging

---

## 🎓 RESUMEN DE CONCEPTOS APRENDIDOS

### Testing
- TestClient para tests rápidos sin servidor
- Fixtures para reutilizar setup
- Mocking con dependency_overrides
- Flujos completos de autenticación

### IA Generativa
- Estructura de llamadas (system/user messages)
- Control de tokens y costes
- Rate limiting con ventanas temporales
- Retry logic con backoff exponencial
- Logging estructurado

### Patrones de producción
- Validación en capas (Pydantic + negocio)
- Manejo robusto de errores
- Métricas de negocio
- Resiliencia y recuperación

**¡Felicitaciones por completar todos los ejercicios! 🎉**